## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-12-22-47-28 +0000,nypost,Trio of MS-13 migrants busted in Nassau County...,https://nypost.com/2025/08/12/us-news/trio-of-...
1,2025-08-12-22-47-27 +0000,nyt,D.C. Officials Discuss Federal Police Takeover...,https://www.nytimes.com/2025/08/12/us/politics...
2,2025-08-12-22-46-50 +0000,startribune,Jury convicts Blaine man in $7M Medicaid fraud...,https://www.startribune.com/jury-convicts-blai...
3,2025-08-12-22-45-04 +0000,nyt,"Under Pressure by Trump, Mexico Sends 26 Accus...",https://www.nytimes.com/2025/08/12/world/ameri...
4,2025-08-12-22-35-45 +0000,nypost,British cops wore jogging outfits to elicit ca...,https://nypost.com/2025/08/12/world-news/femal...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2371/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
33,trump,56
111,up,12
179,china,12
51,report,11
127,will,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
267,2025-08-12-01-47-00 +0000,wsj,President Trump said he would deploy National ...,https://www.wsj.com/politics/policy/trump-dc-n...,118
272,2025-08-12-01-00-00 +0000,wsj,President Trump’s decision to deploy National ...,https://www.wsj.com/politics/policy/trump-nati...,104
280,2025-08-12-00-04-00 +0000,wsj,"President Trump plans to nominate E.J. Antoni,...",https://www.wsj.com/politics/policy/trump-to-n...,101
83,2025-08-12-19-26-00 +0000,wsj,President Trump on Tuesday appeared to call fo...,https://www.wsj.com/economy/trump-calls-on-gol...,100
203,2025-08-12-10-35-02 +0000,nypost,JD Vance says Elon Musk should return to White...,https://nypost.com/2025/08/12/us-news/jd-vance...,99


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
267,118,2025-08-12-01-47-00 +0000,wsj,President Trump said he would deploy National ...,https://www.wsj.com/politics/policy/trump-dc-n...
85,56,2025-08-12-19-21-00 +0000,wsj,The State Department released a long-delayed h...,https://www.wsj.com/politics/policy/state-depa...
10,55,2025-08-12-22-22-09 +0000,nypost,NYC ex-cons slice and dice their way through n...,https://nypost.com/2025/08/12/us-news/nyc-ex-c...
120,50,2025-08-12-17-33-00 +0000,wsj,Crypto Entrepreneur Do Kwon Pleads Guilty to F...,https://www.wsj.com/finance/currencies/crypto-...
203,43,2025-08-12-10-35-02 +0000,nypost,JD Vance says Elon Musk should return to White...,https://nypost.com/2025/08/12/us-news/jd-vance...
279,43,2025-08-12-00-10-09 +0000,nypost,ABC News anchor Kyra Phillips reveals she was ...,https://nypost.com/2025/08/11/us-news/abc-news...
64,39,2025-08-12-20-22-00 +0000,wsj,U.S. Steel Plant Where Two Died Will Continue ...,https://www.wsj.com/us-news/clariton-coke-work...
280,39,2025-08-12-00-04-00 +0000,wsj,"President Trump plans to nominate E.J. Antoni,...",https://www.wsj.com/politics/policy/trump-to-n...
134,36,2025-08-12-16-42-05 +0000,bbc,Israel bombards Gaza City as UK and allies dem...,https://www.bbc.com/news/articles/clyj0dd0qj9o...
132,36,2025-08-12-16-45-07 +0000,nyt,Philippines Condemns China After South China S...,https://www.nytimes.com/2025/08/12/world/asia/...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
